In [1]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187815463 sha256=d944fc7d2f962bce83fc4708c2fc0c21eaf8255962a0b350ae919362a51b7ef2
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash_attn
  Att

In [2]:
import os
import pickle
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from PIL import Image
from transformers import AutoModel, AutoProcessor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Định nghĩa các đường dẫn
BASE_DIR = '/kaggle/input/data-nlp-bai-2/img'
OUTPUT_DIR = '/kaggle/working/'
IMAGE_DIR = os.path.join(BASE_DIR, 'img')
FEATURES_FILE = os.path.join(OUTPUT_DIR, 'internvit_features.pkl')

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Dataset tùy chỉnh để tải ảnh
class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = [
            f for f in os.listdir(image_dir) 
            if f.endswith(('.jpg', '.jpeg', '.png')) and f not in ['6803.jpg', '6801.jpg']  # Skip 6803.jpg and 6801.jpg
        ]
        # Transform để resize và chuyển ảnh thành tensor
        self.transform = transforms.Compose([
            transforms.Resize((448, 448)),  # Resize về 448x448 (phù hợp InternViT)
            transforms.ToTensor(),  # Chuyển PIL Image thành tensor (C, H, W)
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Chuẩn hóa
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image_id = os.path.splitext(img_name)[0]
        # Chuyển ảnh thành tensor
        image_tensor = self.transform(image)
        return image_tensor, image_id  # Chỉ trả về tensor và image_id

# Tải mô hình InternViT và processor
model_name = "OpenGVLab/InternViT-300M-448px-V2_5"
processor = AutoProcessor.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_flash_attn=False
)
model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_flash_attn=False
)

# Chuyển mô hình sang GPU chính (cuda:0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Đảm bảo mô hình nằm trên cuda:0
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model, device_ids=[0, 1])  # Sử dụng cả 2 GPU
model.eval()

# Hàm chuyển tensor về PIL Image
def tensor_to_pil(tensor, device):
    # Denormalize và chuyển tensor về numpy
    mean = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(device)
    std = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(device)
    tensor = tensor * std + mean
    tensor = tensor.clamp(0, 1) * 255
    tensor = tensor.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    return Image.fromarray(tensor)

# Hàm trích xuất đặc trưng cho một batch ảnh
def extract_feature_batch(image_tensors, device):
    # Chuyển batch tensor thành list PIL Image
    pil_images = [tensor_to_pil(tensor, device) for tensor in image_tensors]
    # Xử lý với processor
    inputs = processor(images=pil_images, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        features = outputs.last_hidden_state[:, 0, :]  # Lấy [CLS] token
    
    return features.cpu().numpy()

# Làm sạch bộ nhớ GPU trước khi chạy
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Tạo dataset và dataloader
dataset = ImageDataset(IMAGE_DIR)
batch_size = 16  # Tùy chỉnh batch size tùy theo bộ nhớ GPU
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Dictionary để lưu đặc trưng
features = {}

# Trích xuất đặc trưng với tiến trình
for batch_image_tensors, batch_image_ids in tqdm(dataloader, desc="Extracting features"):
    # Chuyển batch tensor sang cuda:0
    batch_image_tensors = batch_image_tensors.to(device)
    # Trích xuất đặc trưng
    batch_features = extract_feature_batch(batch_image_tensors, device)
    
    # Lưu đặc trưng vào dictionary
    for image_id, feature in zip(batch_image_ids, batch_features):
        features[image_id] = feature[np.newaxis, :]  # Thêm chiều để giữ shape (1, d)

# Lưu đặc trưng vào file pickle
with open(FEATURES_FILE, 'wb') as f:
    pickle.dump(features, f)

print(f"Features extracted and saved to {FEATURES_FILE}")

2025-05-17 13:56:07.855191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747490168.272952      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747490168.392102      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternViT-300M-448px-V2_5:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternViT-300M-448px-V2_5:
- flash_attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternViT-300M-448px-V2_5:
- modeling_intern_vit.py
- flash_attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


model.safetensors:   0%|          | 0.00/608M [00:00<?, ?B/s]

Using 2 GPUs!


Extracting features: 100%|██████████| 3241/3241 [2:35:58<00:00,  2.89s/it]


Features extracted and saved to /kaggle/working/internvit_features.pkl


In [3]:
features

{'31973': array([[-1.8094891, -0.6708715, -1.170884 , ..., -2.3170388, -1.4706868,
          0.9803534]], dtype=float32),
 '30778': array([[-3.4495857, -3.251882 , -2.773597 , ..., -3.7514968, -1.3546109,
          1.7937062]], dtype=float32),
 '19812': array([[-3.6141086, -4.2683616, -3.7664182, ..., -4.1583314, -1.6080027,
          3.1634731]], dtype=float32),
 '22735': array([[-4.2008705, -4.341855 , -3.7815275, ..., -4.5998635, -1.9209266,
          3.2002494]], dtype=float32),
 '38246': array([[-1.7550938 , -0.65918297, -0.8331886 , ..., -2.961776  ,
         -1.5511038 ,  1.4265118 ]], dtype=float32),
 '44504': array([[-2.3948123 , -0.95585483, -1.2785503 , ..., -2.9086852 ,
         -1.8213774 ,  1.1701716 ]], dtype=float32),
 '16916': array([[-3.8325822, -2.6574264, -2.437123 , ..., -3.4898243, -1.6589148,
          2.114989 ]], dtype=float32),
 '39500': array([[-3.3137777, -2.2805676, -2.3464804, ..., -3.5057294, -1.4454463,
          2.0402393]], dtype=float32),
 '7981': arr

In [4]:
print(features['7981'].shape)  # Ví dụ: (1, 768) hoặc (1, 1024)

(1, 1024)


In [5]:
feature = features['7981'].squeeze(0)  # Chuyển từ (1, d) sang (d,)
feature

array([-2.6308208, -1.0887634, -1.6982777, ..., -3.129169 , -1.6831005,
        1.2527919], dtype=float32)